In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import glob
import os
import requests
from shutil import copyfile
import re
import tabula
import random

In [2]:
# data_consulta = "07/01/2021"
data_consulta = "21/01/2021"

In [3]:
CHROMEDRIVER = r'D:\\Dropbox\\FYY\\Trading\\Projects\\Chromedriver\\chromedriver.exe'

In [4]:
options = webdriver.ChromeOptions()
tgt = r"C\\Users\\fyyamazumi\\Downloads\\"
# profile = {"plugins.plugins_list":[{"enabled":False, "name":"Chrome PDF Viewer"}], "download.default_directory":tgt}
profile = {"plugins.plugins_disabled": ["Chrome PDF Viewer"], "download.default_directory":tgt, "plugins.always_open_pdf_externally": True}
options.add_experimental_option("prefs", profile)
browser = webdriver.Chrome(CHROMEDRIVER, options=options)

In [5]:
browser.implicitly_wait(200)

In [6]:
url = "https://sisweb.tesouro.gov.br/apex/f?p=2501:2::::2::&minimal=full&font=opensans"
# url2 = "https://sisweb.tesouro.gov.br/apex/f?p=2501:2::::2::"

In [7]:
browser.get(url)

In [8]:
time.sleep(1)

In [9]:
from selenium.webdriver.common.keys import Keys

In [10]:
content = browser.page_source
soup = BeautifulSoup(content)
links = soup.find_all("a", text='Download')
table = soup.find_all("table")
df = pd.read_html(str(table))[2]
print(df.iloc[1,6])
print(df.iloc[1,2])
print(df.iloc[1,1])

26/01/2021
Nota do Tesouro Nacional série B (NTN-B)
Resultado do Leilão


In [11]:
def checa_data_escolhida(browser, string_dia_escolhido, str_tipo, str_papel_escolhido = 'Letra do Tesouro Nacional (LTN)'):
    disponivel = False
    content = browser.page_source
    soup = BeautifulSoup(content)
    table = soup.find_all("table")
    df = pd.read_html(str(table))[2]
#     data_site = df.iloc[1,6] #data
#     print(df.iloc[1,2]) #papel
#     print(df.iloc[1,1]) #portaria
    contador = 0
    while contador <=9:
        if df.iloc[contador, 6] == string_dia_escolhido: #data
            if df.iloc[contador, 2] == str_papel_escolhido: #papel
                if df.iloc[contador, 1] == str_tipo: #tipo do documento
                    disponivel = True
                    break
        contador+=1
    return disponivel, contador

In [12]:
def latest_modified_file(Caminho_Pasta):
    list_of_files = glob.glob(Caminho_Pasta + '*')  # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file

In [13]:
def get_link_pdf_from_internal_frame(browser, frame_position=0):
    content = browser.page_source
    soup = BeautifulSoup(content)
    frames = soup.find_all("frame")
    link_pdf = frames[0].get('src')
    return link_pdf

In [14]:
def read_pdf_portaria(filename_pdf, papel="LTN", file_dv_unit=r"D:\Dropbox\FYY\Trading\Projects\TN_Leiloes\DV_unit.csv"):
    # Read pdf into DataFrame
    list_dfs_pdf = tabula.read_pdf(input_path=filename_pdf, multiple_tables=True)
    df_aux = list_dfs_pdf[0] # Work on first table
    df_papel = df_aux.loc[df_aux["Unnamed: 0"] == papel]
    df_papel = df_papel.rename(columns={"Unnamed: 0": "Titulo", "Unnamed: 1": "Cod", "Unnamed: 2": "Vcto", "Data base": "Index", "VN na data-": "VN", "Quantidade": "Qtd_mm"})
    df_papel = df_papel.drop(columns=['Cod','Index', 'VN', 'Juros'])
    df_papel = df_papel.set_index(keys=['Vcto'])
    df_papel["Qtd_mm"] = [float(x.replace(".", "")) for x in df_papel["Qtd_mm"]]
    df_papel['Qtd_mm'] = df_papel['Qtd_mm'] / 1000000.0
    df_papel['DV K'] = 0.0
    df_dv_unit = pd.read_csv(file_dv_unit, delimiter=";", decimal=",", index_col=0)
    dv_unit = df_dv_unit.loc[df_papel.index]['DV unit']
    df_papel["DV K"] = df_papel["Qtd_mm"] * dv_unit * 1000.0
    # If not issued for public, zero
    df_papel.loc[df_papel['Adquirente'] != "Público", "DV K"] = 0.0
    return df_papel

In [15]:
checa_data_escolhida(browser, data_consulta, "Portaria", 'Nota do Tesouro Nacional série F (NTN-F)')

(True, 4)

In [16]:
checa_data_escolhida(browser, data_consulta, "Resultado do Leilão", 'Nota do Tesouro Nacional série F (NTN-F)')

(True, 7)

In [17]:
links = soup.find_all("a", text='Download')

In [18]:
resultado_LTN = checa_data_escolhida(browser, data_consulta, "Portaria", "Letra do Tesouro Nacional (LTN)")
resultado_NTNF = checa_data_escolhida(browser, data_consulta, "Portaria", "Nota do Tesouro Nacional série F (NTN-F)")

In [19]:
def procurar_ultimo_arquivo(pasta_procurar, string_procurar):
    # se string for localizada no nome do ultimo arquivo modificado, o find retorna a posição, caso contrário retorna -1.
    localizado = latest_modified_file(pasta_procurar).find(string_procurar) > 0
    return localizado

In [20]:
def get_pdf_portaria(browser, link_position, pdf_filename="D:\\Dropbox\\FYY\\Trading\\Projects\\TN_Leiloes\\LTN.pdf", search_for_str="Portaria"):
    """
    link_position:
        usar resultado_LTN[1] ou resultado_NTNF[1]
    """
    # 1. obtem o URL da portaria (uma pagina que tem um frame com PDF embutido dentro)
    url_Portaria = links[link_position].get('href')
    browser.get(url_Portaria)
    # 2. obtem o URL do PDF dentro do frame embutido na página
    link_pdf_portaria = get_link_pdf_from_internal_frame(browser)
    print(str(link_pdf_portaria))
    # 3. Faz o download do PDF na pasta downloads (default definido nas opçoes do browser)
    browser.get(link_pdf_portaria) 
    # 4. Salva o arquivo gerado numa pasta especificada
    # 4.1 Localiza o arquivo salvo na pasta downloads
    usuario = os.environ['USERNAME']
    src_path = "C:\\users\\" + usuario + "\\Downloads\\"
    str_Procurar = search_for_str #procura arquivo com esta string no nome
    time.sleep(1)
    while procurar_ultimo_arquivo(src_path, str_Procurar) == False:
        time.sleep(1)
    if procurar_ultimo_arquivo(src_path, str_Procurar):
        src = latest_modified_file(src_path)
        print(src)
#         dst = "C:\\Users\\" + usuario + "\\Documents\\My_Projects\\Projetos\\TN_Leilao\\LTN.pdf"
        dst = pdf_filename
        print(dst)
        copyfile(src, dst)
        time.sleep(1)
        #Deleta arquivo da pasta downloads
        os.remove(src)
        print("File Removed from Downloads!")

### Repetidor

```Python
LTN_OK = False
NTNF_OK = False
# Condition to loop
not(NTNF_OK) or not(LTN_OK)
```

In [21]:
LTN_OK = False
NTNF_OK = False

In [22]:
random.seed(a=5)

while not(NTNF_OK):
    botao_pesquisar = browser.find_elements_by_xpath("//button[@id='botao-pesquisar']")
    botao_pesquisar[0].click()
    content = browser.page_source
    soup = BeautifulSoup(content)
    links = soup.find_all("a", text='Download')
#     resultado_LTN = checa_data_escolhida(browser, data_consulta, "Portaria", "Letra do Tesouro Nacional (LTN)")
    resultado_NTNF = checa_data_escolhida(browser, data_consulta, "Portaria", "Nota do Tesouro Nacional série F (NTN-F)")

    time.sleep(1.5 + random.random())
#     if resultado_LTN[0] and not(LTN_OK):
#         LTN_pdf_filename = "D:\\Dropbox\\FYY\\Trading\\Projects\\TN_Leiloes\\LTN.pdf"
#         get_pdf_portaria(browser=browser, 
#                          link_position=resultado_LTN[1], 
#                          pdf_filename=LTN_pdf_filename, 
#                          search_for_str="Portaria")
#         # Read PDF content
#         df_LTN = read_pdf_portaria(filename_pdf=LTN_pdf_filename, papel="LTN", file_dv_unit=r"D:\Dropbox\FYY\Trading\Projects\TN_Leiloes\DV_unit.csv")
#         # Export df
#         df_LTN.to_csv('df_LTN.csv', sep=";", decimal=",", encoding="ANSI")
#         LTN_OK = True
#         browser.get(url)
#         browser.get(url)
    
    if resultado_NTNF[0] and not (NTNF_OK):
        NTNF_pdf_filename = "D:\\Dropbox\\FYY\\Trading\\Projects\\TN_Leiloes\\NTNF.pdf"
        get_pdf_portaria(browser=browser, 
                         link_position=resultado_NTNF[1], 
                         pdf_filename=NTNF_pdf_filename, 
                         search_for_str="Portaria")
        # Read PDF content
        df_NTNF = read_pdf_portaria(filename_pdf=NTNF_pdf_filename, papel="NTN-F", file_dv_unit=r"D:\Dropbox\FYY\Trading\Projects\TN_Leiloes\DV_unit.csv")
        # Export df
        df_NTNF.to_csv('df_NTNF.csv', sep=";", decimal=",", encoding="ANSI")
        NTNF_OK = True
        browser.get(url)
        browser.get(url)
print("NTNF OK!!")

https://cdn.tesouro.gov.br/sistemas-internos///apex//producao//sistemas//thot//arquivos//publicacoes/36886_1204209///1b. Portaria 658.pdf?v=2809
C:\users\Windows\Downloads\1b. Portaria 658.pdf
D:\Dropbox\FYY\Trading\Projects\TN_Leiloes\NTNF.pdf


'pages' argument isn't specified.Will extract only from page 1 by default.


File Removed from Downloads!
NTNF OK!!


In [23]:
browser.quit()